### Prérequis

Pour exécuter le code si joint, il faut installer la librairie jupylet

Voici le guide d'installation

[Guide d'installation](https://jupylet.readthedocs.io/en/latest/programmers_reference_guide/getting_started.html)


```pip install jupylet```

Ensuite il faut procéder à la post-installation.

```python -m jupylet postinstall```

### Pong

Exécutez ce programme et regarder ce qui se passe.

Cliquer la souris dans le l'image du jeu.  Pour bouger les pallettes il faut utiliser les touches flèche gauche et flèche droite pour une palette ainsi que la touche A et D pour l'autre palette


* le son est fournis par  [freesound](https://freesound.org/people/NoiseCollector/sounds/4359/).
* Les fonts de Commodore 64 sont fournis par  [KreativeKorp](https://www.kreativekorp.com/software/fonts/c64.shtml).

Ce jeu est fournis par le site web https://jupylet.readthedocs.io/en/latest/index.html




# Importation des librairies

In [ ]:
import logging
import sys
import os

import numpy as np

In [ ]:
sys.path.insert(0, os.path.abspath('./..'))

Ici on ajoute les librairies de code pour nous permettre d'avoir des fonctionnalités tel que la capacité d'écrire sur l'écran (jupylet.app), d'écrire du texte (jupylet.label), d'afficher des petits morceaux d'image (jupylet.sprite), d'ajouter du son à notre jeu (jupylet.audio.sample)

In [ ]:
import jupylet.color

from jupylet.app import App
from jupylet.state import State
from jupylet.label import Label
from jupylet.sprite import Sprite

from jupylet.audio.sample import Sample

In [ ]:
app = App()#log_level=logging.INFO)

# Création de nos objets du jeu

Ici on défini les couleurs de notre fond d'écran et de l'avant plan ..

In [ ]:
background = '#3e32a2'
foreground = '#7c71da'

Ici on affecte une matrice de 32 * 32 de chiffre 1. Ceci est pour créer une image avec 32 par 32 pixels 

![](.\images\matrix-example2.png)

Par contre la matrice a0 est de grandeur de 32 ligne par 32 colonnes, ce serait assez long ajouter toutes les colonnes et les lignes dans le dessin.

Ensuite on multiplie la matrice par 255, et voilà ce que ça donnera ...

![](.\images\matrix-example3.png)

Encore là, il faut compter 32 lignes et 32 colonnes pour la variable a0

Pour la variable a1, on aurra aussi la valeur 255 dans chacune des cases, cette matrice aura 128 lignes et 16 colonnes qui correspond à la taille en pixel d'une palette.

Enfin la variables a2 est un peu plus compliquée, 

Au lieu d'avoir une matrice, 2 par 2, on aura un cube comme un cube Rubix.

Les trois axes (X, Y, Z) auront différentes valeurs.  
Par example l'axe X sera relié à la largeur 
L'axe Y sera relié à la hauteur de l'application 
L'axe Z représente une profondeur, dans notre cas, c'est pour accumuler la couleur à une position précise.

![](.\images\cube.png)

L'image représente un cube de 5 par 5 par 5.  Dans notre case c'est 90% de la hauteur de l'application par 90% de la largeur de l'application en pixel par 3.


In [ ]:
a0 = np.ones((32, 32)) * 255                                        # génération d'une image avec 32 par 32 pixels pour la balle
a1 = np.ones((128, 16)) * 255                                       # génération d'une image 128 par 16 pixels pour les palettes
a2 = np.ones((app.height * 9 // 10, app.width * 9 // 10, 3)) * 255  # génération des pixels de l'image du fond d'écran, le fond d'écran correspond à 9/10 du jeu, donc le 1/10 de chaque côté sera la bordure 

A cet endroit on définit des image en leur donnant la grandeur qu'on les veut.

Par exemple, **ball** sera la balle qui se déplace.
padl et padr seront les palettes qui se déplacent pour intercepter la balle.

Nous donnons les coordonnées selons les tableaux que nous avons fait préalablement pour indiquer à nos images (Sprite) les positions disponibles dans le notre jeu, ça nous permettra de déplacer les images en vertical (X) et horizontal (y)

field est notre terrain de jeu, l'arrière plan du jeu.

Pour définir l'image, on indique la position dans la variable **x** et **y** comme par exemple, les pallettes **padl** et **palr** sont situés soit près à gauche **48** ou à la droite (**app.width - 48**)

![](.\images\paddle.png)


Pour ce qui est de l'arrière plan du jeu, revenons à l'instruction précédente 
**a2 = np.ones((app.height * 9 // 10, app.width * 9 // 10, 3)) * 255**

Nous indiquons dans le tableau que nous voulons remplir 90% du tableau en hauteur et en largeur, la troisième dimension du tableau permet de conserver la couleur.

Ceci donnera l'effet d'avoir le tour du canvas (10%) avec la couleur de fond. Pour celui-ci on le centre en **x** et **y** dans notre canevas. 

In [ ]:
ball = Sprite(a0, y=app.height/2, x=app.width/2)    # image de la balle

padl = Sprite(a1, y=app.height/2, x=48)             # image de la palette de gauche
padr = Sprite(a1, y=app.height/2, x=app.width-48)   # image pour la palette de droite 

field = Sprite(a2, y=app.height/2, x=app.width/2, color=background)  # image pour l'image de font, l'image mesure 90% du jeu (voir variable a2) et est centré dans l'application, on ajoute la couleur de background pour le sprite 

Comme tout bon jeu, on doit ajouter du son, et pour ça, on lit un fichier de son.  Comme lorsque nous allons dans une application pour écouter de la musique nous choisissons un fichier qui nous fait jouer de la musique.

In [ ]:
pong_sound = Sample('sounds/pong-blip.wav', amp=0.1).load()  # chargement du son

Tout bon jeu à aussi un résultat, il faut l'afficher, donc on utilise un **Label** qui nous permet de visualiser du texte dans notre jeu.  On lui indique des paramètres comme la taille, la position dans notre jeu, le type de lettre.
Le type de lettre est nommé font:
Voici une liste de fonts déjà existants qu'on peut utiliser pour écrire notre texte dans notre jeu.

On peut remarquer que les positions sont centré en vertical **app.height / 2** et horizontal, l'une est positionné à 64 pixels du bord du jeu et l'autre à la droite décalé de 64 pixels, la valeur initiale des **Label** est 0, bien sûr personne n'a encore marqué un point.

Exemple de **fonts**

![](.\images\font.png)

In [ ]:
scorel = Label(
    '0', font_size=42, color=foreground,       # etiquette du résultat à gauche
    x=64, y=app.height/2, 
    anchor_y='center', anchor_x='left',
    font_path='fonts/PetMe64.ttf'
)

scorer = Label(
    '0', font_size=42, color=foreground,      #  etiquette de résultat à droite 
    x=app.width-64, y=app.height/2, 
    anchor_y='center', anchor_x='right',
    font_path='fonts/PetMe64.ttf'
)

# On dessine

Ensuite, il faut bien afficher nos images et notre texte, on utilise la commande **draw** de chacun des textes et des images pour que ceux-ci puissent s'afficher. 

Le premier **field** est le plan de jeu
**scorel** et **scorer** sont les résultats de chacun des joueurs (celui à gauche et celui à droite)

Ensuite on dessine la balle **ball.draw()**

Enfin on dessine les 2 palettes **padl** et **padr**

In [ ]:
@app.event
def render(ct, dt):
    
    app.window.clear(color=foreground)  # on efface tout
    
    field.draw()                        # on dessine le fond du jeu
     
    scorel.draw()                       # on dessine le résultat à gauche
    scorer.draw()                       # on dessine le résultat de droite
    
    ball.draw()                         # on dessine la balle
    padl.draw()                         # on dessine la palette à gauche
    padr.draw()                         # on dessine la palette de droite

#  Résultat d'affichage du jeu sans animation

In [ ]:
import logging
import sys
import os

import numpy as np


sys.path.insert(0, os.path.abspath('./..'))

import jupylet.color

from jupylet.app import App
from jupylet.state import State
from jupylet.label import Label
from jupylet.sprite import Sprite

from jupylet.audio.sample import Sample

app = App()#log_level=logging.INFO)

background = '#3e32a2'
foreground = '#7c71da'

a0 = np.ones((32, 32)) * 255
a1 = np.ones((128, 16)) * 255
a2 = np.ones((app.height * 9 // 10, app.width * 9 // 10, 3)) * 255


pong_sound = Sample('sounds/pong-blip.wav', amp=0.1).load()

ball = Sprite(a0, y=app.height/2, x=app.width/2)

padl = Sprite(a1, y=app.height/2, x=48)
padr = Sprite(a1, y=app.height/2, x=app.width-48)

field = Sprite(a2, y=app.height/2, x=app.width/2, color=background) 

scorel = Label(
    '0', font_size=42, color=foreground, 
    x=64, y=app.height/2, 
    anchor_y='center', anchor_x='left',
    font_path='fonts/PetMe64.ttf'
)

scorer = Label(
    '0', font_size=42, color=foreground, 
    x=app.width-64, y=app.height/2, 
    anchor_y='center', anchor_x='right',
    font_path='fonts/PetMe64.ttf'
)


@app.event
def render(ct, dt):
    
    app.window.clear(color=foreground)
    
    field.draw()
    
    scorel.draw()
    scorer.draw()
    
    ball.draw()
    padl.draw()
    padr.draw()

app.run()

# Initialisation de l'état du jeu

C'est bien d'afficher nos palettes et notre balle mais si elles ne bouge pas alors notre jeu sera très ennuyant.  C'est pour ça qu'on doit ajouter du mouvement et connaître la position de notre balle et de nos pallettes à tout moment pour savoir comment réagir au déplacement de la balle. Ainsi que les touches appuyées par l'utilisateur.

Par exemple lorsque le joueur appuie sur la flèche gauche (left) notre variable d'état **left** sera à vrai (true) et la même chose pour la variable **right** sera à vraie lorsque sa touche sera appuyée, sinon elle sera à **false**.


Maintenant il faut assignée au début du jeu des variables que nous aurons besoin pendant le jeu.


La variable **pyl** permet de regarder la position de la palette à gauche au centre de la palette en vertical, elle est assigné initialement au centre vertical du jeu.

La variable **pyr** elle aussi conserve la position centrale en vertical mais cette fois de la palette a droite,  elle est assigné initialement au centre vertical du jeu.

Les variables **vyl** et **vyr** sont les variables qui conservent la vitesse des palettes et sont assignés à 0 au début car les palettes n'ont pas de vitesse au début du jeu.

Les variables **left, right, key_a et key_d** permettent de savoir si la touche est appuyée au non.  Elle sont assigné à **false** au début car aucune touche n'est appuyé 

Les varaibles **bvx** et **bvy** permettent de conserver la vitesse de la balle en vertical **bvx** et en horizontal **bvy**.  Arbitrairement la vitesse initiale à été choisie à 192 au départ.

Les variables **sl** et **sr** conserve le résultat des joueurs de gauche **sl** et de droite **sr**, elles sont assignées à 0 car elle n'ont aucun pointage.



In [ ]:
state = State(
    
    sl = 0,                             # résultat de gauche
    sr = 0,                             # résultat de droite
    
    bvx = 192,                          # vitesse de la balle horizontale
    bvy = 192,                          # vitesse de la balle verticale
    
    vyl = 0,                            # vitesse de la palette de gauche en vertical
    pyl = app.height/2,                 # position de la palette de gauche 

    vyr = 0,                            # vitesse de la palette de droite en vertical
    pyr = app.height/2,                 # position de la palette de droite en vertical

    left = False,                       # flèche gauche appuyée on non
    right = False,                      # flèche droite appuyée on non

    key_a = False,                      # touche A appuyée ou non
    key_d = False,                      # touche D appuyée ou non
)

# Etat des touches du clavier

Ici on vérifie si une touche est appuyé ou non, on reçoit un événement **keys.ACTION_PRESS** et **keys.ACTION_RELEASE** qui indiquent respectivement, si la touche est enfoncé ou non.  On ajuste alors les variables d'état en conséquence **left, right, key_a, key_d**

In [ ]:
@app.event
def key_event(key, action, modifiers):
        
    keys = app.window.keys
    
    if action == keys.ACTION_PRESS:     # événement d'une touche appuyée
        
        if key == keys.LEFT:            # flèche gauche
            state.left = True

        if key == keys.RIGHT:           # flèche droite
            state.right = True

        if key == keys.A:               # touche A
            state.key_a = True

        if key == keys.D:               # touche D
            state.key_d = True

    if action == keys.ACTION_RELEASE:   # événement d'une touche libérée

    
        if key == keys.LEFT:
            state.left = False

        if key == keys.RIGHT:
            state.right = False

        if key == keys.A:
            state.key_a = False

        if key == keys.D:
            state.key_d = False

# Le déplacement des palettes 

La fonction qui suit est appelée à chaque 1/60 ième de seconde.  Donc c'est comme une horloge qui appelle la fonction.  L'objectif de la fonction est d'évaluer la position des deux palettes à des moments précis pour les dessigner aux endroits appropriés.

### Validation du dépassement des palettes du jeu

Les premières instructions s'assurent que les 2 palettes restent dans le cadre du jeu.
Pour ce faire, on regarde le minimum de la hauteur du jeu et de la position de la palette, on effectue le déplacement de la palette, 
**state.pyr + dt * 512**, en regardant la dernière position de la palette **state.pyr** on ajoute la fraction de temps passée **dt** multiplié par le déplacement de 512 (valeur arbitraire calculée pour une seconde). 
Si on fait par exemple 1/60 seconde * 512  = 8.53 le déplacement dans le temps de 1 soixantième de seconde.  Si le calcul (ancienne position + le calcul précédent) dépasse l'étendu du jeu alors on bloque le déplacement à la grandeur du jeu pas plus autant en haut qu'en bas. 

Les variables **state.right**, **state.left**, **state.key_a**, **state.key_d** indiquent si la touche est appuyer ou non. Si le touches ne sont pas appuyées, c'est grâce aux touches qu'on effectue un déplacement de la palette.


```
    if state.right:
        state.pyr = min(app.height, state.pyr + dt * 512)
        
    if state.left:
        state.pyr = max(0, state.pyr - dt * 512)
        
    if state.key_a:
        state.pyl = min(app.height, state.pyl + dt * 512)
        
    if state.key_d:
        state.pyl = max(0, state.pyl - dt * 512)
```

    Ces lignes ci-haut nous donnerons un nouveau positionnement des palettes seulement si une touche était enfoncée, dans le cas contraire, la position restera la même.

        
### Calcul de l'accélération

    Ici on calcul l'accélération des palettes.  Si une touche n'était pas appuyée alors l'accélération sera constante, on ajoute une constante à l'équation de façon arbitraire (dans ce cas **5**) car la distance est très petite et plus qu'on augmente le nombre plusque l'accélération sera grande.  

![](.\images\acceleration.png)

    [Comprendre l'accélération](https://www.alloprof.qc.ca/fr/eleves/bv/physique/l-acceleration-p1082)


    Si la valeur n'est pas 0 donc il y a eu une vitesse, donc un déplacement dans le dernier 1/60 ième de seconde **state.pyl - padl.y**

    On fait un calcul de déplacement entre la distance du parcourue entre le dernier dessin et la position au temps précis, on peut ajouter un certain facteur car les valeurs numériques sont très petites. Dans ce cas nous avons mis arbitrairement **10**, mais on aurait pu mettre une autre valeur plus grande ou plus petite.      


```
    ayl = 10 * (state.pyl - padl.y)
    ayr = 10 * (state.pyr - padr.y)
```

    Ceci sera notre calcul d'accélération de la palette gauche **ayl** ainsi que de la palette droite **ayr**. Si il n'y a pas de différence entre l'ancienne position de la palette et le dernier état, nous aurons 0 donc une accélération de 0.  Ceci arrivera lorsque qu'aucune touche ne sera appuyée, regarder le code au début de la fonction pour voir quand la variable **state.pyl** et **state.pyr** sera modifié.



### Calcul de la vitesse du dernier laps de temps

```
 (ayl * dt)
 (ayr * dt)
```

    
    L'accélération est définie en distance par seconde carré.  Pour trouver la vitesse si nous mutiplions la l'accélération par le temps, nous avons avoir notre composant vitesse (en distance par secondes)
    Voici la formule de la vitesse  : **v = a*t**


    L'utilisation de la variable **dt** nous permet d'avoir le temps, de faire le rapport entre le temps passé depuis le dernier dessin, donc si nous passons plus de temps entre les intervalles, nous nous déplacerons plus.
    
    Alors notre calcul de vitesse sera **v = a * dt**

### Calcul de la vitesse totale 

```
    state.vyl = state.vyl * 0.9 + (ayl * dt)
    state.vyr = state.vyr * 0.9 + (ayr * dt)
```

    On remarque que lorsque nous lachons la touche d'accélération de la palette, la palette ne s'arrête pas mais diminue de vitesse.

    Ici nous calculons la vitesse de la palette, à chaque fois que la fonction est appelée (1/60 ième) seconde, la vitesse précédente sera  diminuer de 10%, 90% de celle-ci, donc on obtiendra 90% de la vitesse initiale. Ce qui veut dire qu'à chaque cycle, la vitesse diminuera une peu plus.  Nous pouvons jouer sur cette variable et décélérer la palette plus rapidement en mettant 0.5 (50% de facteur de décélération) par exemple.


    Le fait d'utiliser la variable **dt** nous assure que notre calcul sera constant peu importe le notre d'appel à la fonction pour un temps donnée.  Par exemple si nous appelons la fonction 2 fois par seconde (1/2), la vitesse  sera divisée par 2 de la distance totale d'une seconde.  Dans notre cas, la vitesse se sépare 60 fois dans une seconde mais va tout de même arriver à la même vitesse à la fin de la seconde.   


    Le but ici est de calculer la vitesse que nous sommes rendu actuellement, si la variable **state.pyl** a augmenté car une clé était appuyée alors l'accélération sera plus grande donc par le fait même la vitesse sera aussi augmentée, elle ne sera pas constante.

    Nous calculons le nouveau composant de vitesse **(ayl * dt)** et nous l'ajoutons à la vitesse que nous avons déjà
    
    Voici la formule de la vitesse : 
    v = v0 + a*t
    vitesse = ancienne vitesse + nouvelle vitesse

    Dans notre cas v0 (ancienne vitesse) est **state.vyl**, elle avait été calculée le cycle précédent, on peut ajouter la nouvelle composante de vitesse.
    
    Par contre, on sait très bien que la vitesse n'est pas stable dans la nature, il y a la gravité qui la retient.  Pour aider un peut le jeu on diminue la vitesse précédente à 90% de la vitesse initiale et on ajoute le nouveau composant de vitesse.  Ceci permettra de faire ralentir un petit peut à la fois la vitesse jusqu'à en arriver au point d'arrêt si on appuie sur aucune touche.

    Vitesse   = Vitesse * 90%   + Nouvelle vitesse 

```
    state.vyl = state.vyl * 0.9 + (ayl * dt)
    state.vyr = state.vyr * 0.9 + (ayr * dt)
```

    Il ne faut pas oublier que la vitesse peut être négative **state.vyl** et **state.vyr**, donc on se déplace soit vers le bas ou vers le haut.


### Déplacement de la palette

    Une fois qu'on a notre vitesse résultante, on peut calculer la position de la palette en donnant un nouveau centre à notre image (Sprite) qui est le déplacement de son dernier dessin avec la vitesse de ce cycle.

    Formule Distance = Vitesse (m/s) * temps (s).
    Dans notre cas, on remplace les mètres par des pixels

    Distance de la palette est la position précédente + la distance ajoutée dans ce cycle.

    Donc on repositionne l'image de la palette avec sa nouvelle position en **y** en utilisant le composant vitesse qu'on a calculé. 

```
    padl.y += state.vyl * dt
    padr.y += state.vyr * dt
```
    

    A ce moment, on protège le fait de ne pas dessiner les palettes en dehors du cadre du jeu.

```
    padr.clip_position(app.width, app.height)
    padl.clip_position(app.width, app.height)   
```



In [ ]:
@app.run_me_every(1/60)
def update_pads(ct, dt):
        
    if state.right:
        state.pyr = min(app.height, state.pyr + dt * 512)   # assignation de la position de la palette de droite lorsque le déplacement est vers la droite (haut)
        
    if state.left:
        state.pyr = max(0, state.pyr - dt * 512)            # assignation de la position de la palette de droite lorsque le déplacement est vers la gauche (bas)
        
    if state.key_a:
        state.pyl = min(app.height, state.pyl + dt * 512)   # assignation de la position de la palette de gauche lorsque le déplacement est vers la gauche (bas)
        
    if state.key_d:
        state.pyl = max(0, state.pyl - dt * 512)            # assignation de la position de la palette de gauche lorsque le déplacement est vers la droite (haut)
        
    ayl = 10 * (state.pyl - padl.y)                         # calcul de l'accélération de la palette gauche
    ayr = 10 * (state.pyr - padr.y)                         # calcul de l'accélération de la palette droite

    
    state.vyl = state.vyl * 0.9 + (ayl * dt)                # calcul de la vitesse de la palette gauche
    state.vyr = state.vyr * 0.9 + (ayr * dt)                # calcul de la vitesse de la palette droite
    
    
    padl.y += state.vyl * dt                                # calcul de la position de la palette gauche
    padr.y += state.vyr * dt                                # calcul de la position de la palette droite
    
    padr.clip_position(app.width, app.height)               # Conserver l'image de la palette droite dans le cadre du jeu
    padl.clip_position(app.width, app.height)               # Conserver l'image de la palette gauche dans le cadre du jeu

# Jeu avec déplacement des palettes

In [ ]:
import logging
import sys
import os

import numpy as np


sys.path.insert(0, os.path.abspath('./..'))

import jupylet.color

from jupylet.app import App
from jupylet.state import State
from jupylet.label import Label
from jupylet.sprite import Sprite

from jupylet.audio.sample import Sample

app = App()#log_level=logging.INFO)

background = '#3e32a2'
foreground = '#7c71da'

a0 = np.ones((32, 32)) * 255
a1 = np.ones((128, 16)) * 255
a2 = np.ones((app.height * 9 // 10, app.width * 9 // 10, 3)) * 255


pong_sound = Sample('sounds/pong-blip.wav', amp=0.1).load()

ball = Sprite(a0, y=app.height/2, x=app.width/2)

padl = Sprite(a1, y=app.height/2, x=48)
padr = Sprite(a1, y=app.height/2, x=app.width-48)

field = Sprite(a2, y=app.height/2, x=app.width/2, color=background) 

scorel = Label(
    '0', font_size=42, color=foreground, 
    x=64, y=app.height/2, 
    anchor_y='center', anchor_x='left',
    font_path='fonts/PetMe64.ttf'
)

scorer = Label(
    '0', font_size=42, color=foreground, 
    x=app.width-64, y=app.height/2, 
    anchor_y='center', anchor_x='right',
    font_path='fonts/PetMe64.ttf'
)


@app.event
def render(ct, dt):
    
    app.window.clear(color=foreground)
    
    field.draw()
    
    scorel.draw()
    scorer.draw()
    
    ball.draw()
    padl.draw()
    padr.draw()


state = State(
    
    sl = 0,
    sr = 0,
    
    bvx = 192,
    bvy = 192,
    
    vyl = 0,
    pyl = app.height/2,

    vyr = 0,
    pyr = app.height/2,

    left = False,
    right = False,

    key_a = False,
    key_d = False,
)    


@app.event
def key_event(key, action, modifiers):
        
    keys = app.window.keys
    
    if action == keys.ACTION_PRESS:
        
        if key == keys.LEFT:
            state.left = True

        if key == keys.RIGHT:
            state.right = True

        if key == keys.A:
            state.key_a = True

        if key == keys.D:
            state.key_d = True

    if action == keys.ACTION_RELEASE:

    
        if key == keys.LEFT:
            state.left = False

        if key == keys.RIGHT:
            state.right = False

        if key == keys.A:
            state.key_a = False

        if key == keys.D:
            state.key_d = False


@app.run_me_every(1/60)
def update_pads(ct, dt):
        
    if state.right:
        state.pyr = min(app.height, state.pyr + dt * 512)
        
    if state.left:
        state.pyr = max(0, state.pyr - dt * 512)
        
    if state.key_a:
        state.pyl = min(app.height, state.pyl + dt * 512)
        
    if state.key_d:
        state.pyl = max(0, state.pyl - dt * 512)
        
    ayl = 10 * (state.pyl - padl.y)
    ayr = 10 * (state.pyr - padr.y)

    
    state.vyl = state.vyl * 0.9 + (ayl * dt)
    state.vyr = state.vyr * 0.9 + (ayr * dt)
    
    
    padl.y += state.vyl * dt
    padr.y += state.vyr * dt
    
    padr.clip_position(app.width, app.height)
    padl.clip_position(app.width, app.height)


app.run()


# Déplacement de la balle 

Maintenant c'est le tour de la balle,  
Il faut déplacer la balle dans un intervalle précis, encore une fois tous les 1 soixantième de secondes.

Ici on conserve la composant vitesse cumulée de la balle (horizontale et verticale) pour une comparaison future.

**bs0 = state.bvx ** 2 + state.bvy ** 2**

    Comme la balle qui est un carré tourne toujours, On calcule une portion de la rotation de l'angle selon le temps (1/60) donc l'angle sera 200 degré par seconde, on prend une part de cet angle à tous les 1/60 ième de seconde.  

    ball.angle += 200 * dt

    
    Comme la palette précédemment, la balle a une vitesse mais cette fois la vitesse est sous forme horizontale et verticales contrairement à la palette qui avant seulement une vitesse verticale.

    Donc le déplacement de la balle se calcule par sa position précédente + la vitesse * par le temps.

    **nouvelle position = position actuelle + vitesse * temps**

    On calcule le déplacement de la balle en horizontal selon la vitesse de la balle

    Celle-ci est la position de la balle en horizontale recalculé avec le nouveau cycle
    **ball.x += state.bvx * dt**

    Celle-ci est la position de la balle en verticale recalculé avec le nouveau cycle
    **ball.y += state.bvy * dt**
    
### Balle trop haute
    Si la balle atteint le haut du jeu, on doit la retourner dans le jeu en calculant l'angle de retour, on ramène la balle à la position maximum du jeu (pas à l'extérieur) on inverse sa velocité de la balle pour qu'elle descende ** ball.top >= app.height**
    Cette instuction permet de jouer un son.
    ```
    pong_sound.play(pan=2*max(.25, min(.75, ball.x / app.width))-1)
    ```

```
    if ball.top >= app.height:
        pong_sound.play(pan=2*max(.25, min(.75, ball.x / app.width))-1)
        ball.y -= ball.top - app.height
        state.bvy = -state.bvy
```

### Balle trop basse
    Si la balle est plus basse que le jeu **ball.bottom <= 0**
    Si la balle a atteint le bas du jeu, on la retourne comme ci-haut en inversant la vitesse verticale de la balle **bvy** pour qu'elle remonte

```
    if ball.bottom <= 0:
        pong_sound.play(pan=2*max(.25, min(.75, ball.x / app.width))-1)
        ball.y -= ball.bottom
        state.bvy = -state.bvy
```

### Balle trop à droite

    Si la balle dépasse à droite, on ramène la composant horizontal de la balle (x) dans le jeu, ensuite il faut changer la vélocité de X et de Y car la balle va suivre un angle inverse (elle rebondit)    

    On va inversé la vélocité horizontale (x) car la balle passe de vers la droite à vers la gauche.  Pour la vélocité verticale, on garde la même direction.  Donc la vitesse choisie (192 valeur arbitraire choisie initialement au début du jeu) conserve le même signe en verticale et change de signe en horizontal. 

```
    if ball.right >= app.width:
        pong_sound.play(pan=2*max(.25, min(.75, ball.x / app.width))-1)
        ball.x -= ball.right - app.width
        
        state.bvx = -192
        state.bvy = 192 * np.sign(state.bvy)
        bs0 = 0

```

    Enfin, il faut aussi donner un point à l'autre joueur, celui de gauche, si la balle se rend à droite complètement, **state.sl** et changer le texte du label **scorel**

```
        state.sl += 1
        scorel.text = str(state.sl)
```

### Balle trop à gauche

    Pour la gauche, c'est pratiquement le même code, si la balle dépasse à droite, on ramène le composant horizontal de la balle (x) dans le jeu. On inverse le signe en horizontale de la vélocité de la balle (bvx) et on garde le même signe en vertical de la vélocité de la balle (bvy), là-aussi le pointage augmente, mais cette fois-ci, pour l'autre joueur.

```   
    if ball.left <= 0:
        pong_sound.play(pan=2*max(.25, min(.75, ball.x / app.width))-1)
        ball.x -= ball.left
        
        state.bvx = 192
        state.bvy = 192 * np.sign(state.bvy)
        bs0 = 0
        
        state.sr += 1
        scorer.text = str(state.sr)
```   
 
### Balle proche de la palette droite 

 Si la balle est à la même hauteur que la palette **ball.top >= padr.bottom and padr.top >= ball.bottom**
 On vérifie si la distance entre la balle et la palette est sous 10 pixels **0 < ball.right - padr.left < 10**

    Ici on va regarder si la palette est proche de la balle du côté droit du jeu.   Si c'est le cas, on repositionne la balle pour la coller sur la pallette, on inverse la vitesse horizontale de la balle **bvx** et on augmente la vitesse de la balle de 50% de la vitesse de la palette. Ceci est pour simuler qu'un effet de vitesse augmente la vitesse de la balle, dans la réalité, c'est plutôt une vitesse de rotation de la balle qui augmenterait et on aurait un effet de courbe comme lorsque l'on joue au squash et au tennis. 

```   
    if state.bvx > 0 and ball.top >= padr.bottom and padr.top >= ball.bottom: 
        if 0 < ball.right - padr.left < 10:
            pong_sound.play(pan=2*max(.25, min(.75, ball.x / app.width))-1)
            ball.x -= ball.right - padr.left
            state.bvx = -state.bvx
            state.bvy += state.vyr / 2
```   
             

### Balle proche de la palette gauche 

    Maintenant on s'occupe du côté gauche du jeu et on fait sensiblement le même traitement pour la balle.  
```   
    if state.bvx < 0 and ball.top >= padl.bottom and padl.top >= ball.bottom: 
        if 0 < padl.right - ball.left < 10:
            pong_sound.play(pan=2*max(.25, min(.75, ball.x / app.width))-1)
            ball.x += ball.left - padl.right
            state.bvx = -state.bvx
            state.bvy += state.vyl / 2
```   
 
### Repositionnement de la vitesse de la balle en horizontal

    Maintenant on recalcule la vitesse totale en horizontal et vertical, le même calcul qu'on avant fait au début de la fonction.        
            
 ```   
    bs1 = state.bvx ** 2 + state.bvy ** 2
 ```
    Ici on ajuste la vitesse de la balle horizontale si elle est trop basse par rapport au début de la fonction

    Si le nouveau calcul de vitesse est plus petit de 90% de la vitesse avant le réajustement de la vitesse dans cette fonction
    On recalcule la vitesse en horizontal sans tenir compte de la coposante de vitess verticale **state.bvy*
    En gros on ramène la vitesse horizontale **bvx** à son origine avant l'exécution de la fonction 

    if bs1 < 0.9 * bs0:
        state.bvx = (bs0 - state.bvy ** 2) ** 0.5 * np.sign(state.bvx)



    Enfin, on s'assure que la balle reste dans le cadre du jeu
    ball.wrap_position(app.width, app.height)

In [1]:
@app.run_me_every(1/60)
def update_ball(ct, dt):
    
    bs0 = state.bvx ** 2 + state.bvy ** 2    # premier calcul de la vitesse horizontale et verticale cumulée
    
    ball.angle += 200 * dt                   # changement de l'angle de la balle (carrée) en proportion de 1/60 seconde 
    
    ball.x += state.bvx * dt                 # déplacement horizontal de la balle sur 1/60 seconde
    ball.y += state.bvy * dt                 # déplacement vertical de la balle sur 1/60 seconde   
    
    if ball.top >= app.height:                                              # vérification si la balle est trop haute 
        pong_sound.play(pan=2*max(.25, min(.75, ball.x / app.width))-1)     # on joue une note  
        ball.y -= ball.top - app.height                                     # on reposition la balle dans la limite du jeu
        state.bvy = -state.bvy                                              # on inverse la vitesse pour faire un changement de direction en vertical seulement
        
    if ball.bottom <= 0:                                                    # vérification si la balle est trop basse
        pong_sound.play(pan=2*max(.25, min(.75, ball.x / app.width))-1)     # on joue une note     
        ball.y -= ball.bottom                                               # on ramette la balle dans les limites du jeu
        state.bvy = -state.bvy                                              # on inverse la vitesse pour faire un changement de direction en vertical seulement    
        
    if ball.right >= app.width:                                             # vérification si la balle est trop à droite
        pong_sound.play(pan=2*max(.25, min(.75, ball.x / app.width))-1)     # on joue une note
        ball.x -= ball.right - app.width                                    # on repositionne la balle dans le cadre du jeu
        
        state.bvx = -192                                                    # on change la direction horizontale de la vitesse de la balle et on la met à 192
        state.bvy = 192 * np.sign(state.bvy)                                # on assigne la vitesse de la balle à 192 sans changer la direction en verticale
        bs0 = 0                                                             # on met la vitesse initiale cumulée a 0 lorsque la limite du jeu est atteinte
        
        state.sl += 1                                                       # on donne un point au joueur de gauche car la balle a atteint la limite droite
        scorel.text = str(state.sl)                                         # on affiche le score sur le plateau de jeu du joueur de gauche    
        
    if ball.left <= 0:                                                  
        pong_sound.play(pan=2*max(.25, min(.75, ball.x / app.width))-1)             # On vérifie si la balle est trop à gauche
        ball.x -= ball.left                                                         # on reposition la balle dans la limite du jeu
        
        state.bvx = 192                                                             # on applique la vitesse horizontale de la balle à la valeur par défaut en positif pour que la direction soit vers la droite. 
        state.bvy = 192 * np.sign(state.bvy)                                        # on applique la vitesse verticale de la balle à la valeur par défaut sans changer le signe de la direction verticale
        bs0 = 0                                                                     # on met la vitesse initiale cumulée a 0 lorsque la limite du jeu est atteinte
        
        state.sr += 1                                                               # on donne un point au joueur de droite car la balle a toucher a la limite gauche
        scorer.text = str(state.sr)
        
    if state.bvx > 0 and ball.top >= padr.bottom and padr.top >= ball.bottom:       # on vérifie si la balle est dans les limite verticale de la palette de droite
        if 0 < ball.right - padr.left < 10:                                         # on vérifie si la balle est 10 pixels ou moins près de la palette de droite     
            pong_sound.play(pan=2*max(.25, min(.75, ball.x / app.width))-1)         # on joue un son
            ball.x -= ball.right - padr.left                                        # on position la balle sur la palette de droite
            state.bvx = -state.bvx                                                  # on inverse la vitesse horizontale de la balle pour faire un changement de direction
            state.bvy += state.vyr / 2                                              # on ajuste la vitesse verticale de la balle en lui ajoutant 50% de la vitesse de la palette droite
            
    if state.bvx < 0 and ball.top >= padl.bottom and padl.top >= ball.bottom:       # on vérifie si la balle est dans les limite verticale de la palette de gauche
        if 0 < padl.right - ball.left < 10:                                         # on vérifie si la balle est 10 pixels ou moins près de la palette de gauche
            pong_sound.play(pan=2*max(.25, min(.75, ball.x / app.width))-1)         # on joue un son    
            ball.x += ball.left - padl.right                                        # on position la balle sur la palette de gauche
            state.bvx = -state.bvx                                                  # on inverse la vitesse horizontale de la balle pour faire un changement de direction
            state.bvy += state.vyl / 2                                              # on ajuste la vitesse verticale de la balle en lui ajoutant 50% de la vitesse de la palette droite
            
    bs1 = state.bvx ** 2 + state.bvy ** 2                                           # deuxième calcul de la vitesse horizontale et verticale cumulée
    
    if bs1 < 0.9 * bs0:                                                             # Si le deuxième calcul est plus petit que 90% de la vitesse cumulée, on recalcul la vitesse de la balle en horizontale pour remettre la vitesse horizontale initiale
        state.bvx = (bs0 - state.bvy ** 2) ** 0.5 * np.sign(state.bvx)              

    ball.wrap_position(app.width, app.height)                                       # on s'assure de ne pas dessiner la balle hors des limites du jeu    

NameError: name 'app' is not defined

# Maintenant, executons le programme avec la balle qui bouge

In [2]:
import logging
import sys
import os

import numpy as np


sys.path.insert(0, os.path.abspath('./..'))

import jupylet.color

from jupylet.app import App
from jupylet.state import State
from jupylet.label import Label
from jupylet.sprite import Sprite

from jupylet.audio.sample import Sample

app = App()#log_level=logging.INFO)

background = '#3e32a2'
foreground = '#7c71da'

a0 = np.ones((32, 32)) * 255
a1 = np.ones((128, 16)) * 255
a2 = np.ones((app.height * 9 // 10, app.width * 9 // 10, 3)) * 255


pong_sound = Sample('sounds/pong-blip.wav', amp=0.1).load()

ball = Sprite(a0, y=app.height/2, x=app.width/2)

padl = Sprite(a1, y=app.height/2, x=48)
padr = Sprite(a1, y=app.height/2, x=app.width-48)

field = Sprite(a2, y=app.height/2, x=app.width/2, color=background) 

scorel = Label(
    '0', font_size=42, color=foreground, 
    x=64, y=app.height/2, 
    anchor_y='center', anchor_x='left',
    font_path='fonts/PetMe64.ttf'
)

scorer = Label(
    '0', font_size=42, color=foreground, 
    x=app.width-64, y=app.height/2, 
    anchor_y='center', anchor_x='right',
    font_path='fonts/PetMe64.ttf'
)


@app.event
def render(ct, dt):
    
    app.window.clear(color=foreground)
    
    field.draw()
    
    scorel.draw()
    scorer.draw()
    
    ball.draw()
    padl.draw()
    padr.draw()


state = State(
    
    sl = 0,
    sr = 0,
    
    bvx = 192,
    bvy = 192,
    
    vyl = 0,
    pyl = app.height/2,

    vyr = 0,
    pyr = app.height/2,

    left = False,
    right = False,

    key_a = False,
    key_d = False,
)    


@app.event
def key_event(key, action, modifiers):
        
    keys = app.window.keys
    
    if action == keys.ACTION_PRESS:
        
        if key == keys.LEFT:
            state.left = True

        if key == keys.RIGHT:
            state.right = True

        if key == keys.A:
            state.key_a = True

        if key == keys.D:
            state.key_d = True

    if action == keys.ACTION_RELEASE:

    
        if key == keys.LEFT:
            state.left = False

        if key == keys.RIGHT:
            state.right = False

        if key == keys.A:
            state.key_a = False

        if key == keys.D:
            state.key_d = False


@app.run_me_every(1/60)
def update_pads(ct, dt):
        
    if state.right:
        state.pyr = min(app.height, state.pyr + dt * 512)
        
    if state.left:
        state.pyr = max(0, state.pyr - dt * 512)
        
    if state.key_a:
        state.pyl = min(app.height, state.pyl + dt * 512)
        
    if state.key_d:
        state.pyl = max(0, state.pyl - dt * 512)
        
    ayl = 10 * (state.pyl - padl.y)
    ayr = 10 * (state.pyr - padr.y)

    
    state.vyl = state.vyl * 0.9 + (ayl * dt)
    state.vyr = state.vyr * 0.9 + (ayr * dt)
    
    
    padl.y += state.vyl * dt
    padr.y += state.vyr * dt
    
    padr.clip_position(app.width, app.height)
    padl.clip_position(app.width, app.height)



@app.run_me_every(1/60)
def update_ball(ct, dt):
    
    bs0 = state.bvx ** 2 + state.bvy ** 2
    
    ball.angle += 200 * dt
    
    ball.x += state.bvx * dt
    ball.y += state.bvy * dt
    
    if ball.top >= app.height:
        pong_sound.play(pan=2*max(.25, min(.75, ball.x / app.width))-1)
        ball.y -= ball.top - app.height
        state.bvy = -state.bvy
        
    if ball.bottom <= 0:
        pong_sound.play(pan=2*max(.25, min(.75, ball.x / app.width))-1)
        ball.y -= ball.bottom
        state.bvy = -state.bvy
        
    if ball.right >= app.width:
        pong_sound.play(pan=2*max(.25, min(.75, ball.x / app.width))-1)
        ball.x -= ball.right - app.width
        
        state.bvx = -192
        state.bvy = 192 * np.sign(state.bvy)
        bs0 = 0
        
        state.sl += 1
        scorel.text = str(state.sl)
        
     # + ' ' + state.pyr + ' ' + padl.y    
    if ball.left <= 0:
        pong_sound.play(pan=2*max(.25, min(.75, ball.x / app.width))-1)
        ball.x -= ball.left
        
        state.bvx = 192
        state.bvy = 192 * np.sign(state.bvy)
        bs0 = 0
        
        state.sr += 1
        scorer.text = str(state.sr)
        
    if state.bvx > 0 and ball.top >= padr.bottom and padr.top >= ball.bottom: 
        if 0 < ball.right - padr.left < 10:
            pong_sound.play(pan=2*max(.25, min(.75, ball.x / app.width))-1)
            ball.x -= ball.right - padr.left
            state.bvx = -state.bvx
            state.bvy += state.vyr / 2
            
    if state.bvx < 0 and ball.top >= padl.bottom and padl.top >= ball.bottom: 
        if 0 < padl.right - ball.left < 10:
            pong_sound.play(pan=2*max(.25, min(.75, ball.x / app.width))-1)
            ball.x += ball.left - padl.right
            state.bvx = -state.bvx
            state.bvy += state.vyl / 2
            
    bs1 = state.bvx ** 2 + state.bvy ** 2
    
    if bs1 < 0.9 * bs0:
        state.bvx = (bs0 - state.bvy ** 2) ** 0.5 * np.sign(state.bvx)

    ball.wrap_position(app.width, app.height)


app.run()

Image(value=b'\xff\xd8\xff\xe0\x00\x10JFIF\x00\x01\x01\x00\x00\x01\x00\x01\x00\x00\xff\xdb\x00C\x00\x08\x06\x0…

# Voici tout le programme

Image(value=b'\xff\xd8\xff\xe0\x00\x10JFIF\x00\x01\x01\x00\x00\x01\x00\x01\x00\x00\xff\xdb\x00C\x00\x08\x06\x0…

# Exercice 1

Arrêter la balle de tournée



# Exercice 2

Mettre le score en bas du jeu

# Exercice 3

Arrêter le jeu à 10 en indiquant le vainqueur. On efface la balle et les palettes.


# Exercice 4

Diminuer l'accélération de des palettes lorsqu'on appuie sur les touches.